In [ ]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
import pytz


base_url = 'https://www.casa.it/vendita/residenziale/trieste?page={}'


AllHouseOutput = []
def scrape_data():
    url = base_url.format(1)
    req = requests.get(url).text
    soup = BeautifulSoup(req, 'html.parser')

    
    #we can assume total pages is 45 for now
    #total_pages = (int(soup.find('h1', class_='heading__h1')['data-count'].strip()) + 19) // 20
    
    total_pages = 45

    for page_number in range(1, total_pages + 1):
        url = base_url.format(page_number)
        req = requests.get(url).text
        soup = BeautifulSoup(req, 'html.parser')
        house_list = soup.find_all('div', class_='art-infos is-clickable')

        for house in house_list:
            title_tag = house.find('p', class_='art-addr__txt')
            title = title_tag.a.text.strip() if title_tag and title_tag.a else None

            area_tag = house.find('p', class_='art-addr__addrs-subt')
            area = area_tag.text.strip() if area_tag else None

            price_tag = house.find('div', class_='info-features__price')
            price = price_tag.p.text.strip().replace('€', '').replace('.', '').strip() if price_tag and price_tag.p else None

            mq_tag = house.find('div', class_='info-features__item', text=lambda x: 'mq' in x)
            mq = mq_tag.text.strip().split()[0] if mq_tag else None

            locali_tag = house.find('div', class_='info-features__item', text=lambda x: 'locali' in x)
            locali = locali_tag.text.strip().split()[0] if locali_tag else None

            description_tag = house.find('div', class_='art-desc__content', class_=lambda x: x and ('open' in x or 'closed' in x))
            description = description_tag.text.strip() if description_tag else None

            House_details = {
                'Titolo': title,
                'Area': area,
                'Price': price,
                'MQ': mq,
                'Locali': locali,
                'Description': description
            }


            AllHouseOutput.append(House_details)
    

    return AllHouseOutput


scrape_data()